In [40]:
num_epochs = 10
batch_size = 32
best_accuracy = 50
#num_layers = 9

In [41]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [42]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'discard']  # 排除 'discard' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        if 'discard' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'Discard' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values.astype("float32")
        train_data = train_data.reshape(34, 29)
        
        value_data = df['discard'].iloc[sample_idx]  # 提取 'Discard' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.long)
        
    def __len__(self):
        return self.num_samples

In [43]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2021/DiscardData.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

# 測試 DataLoader 是否正常運作
for batch_features, batch_labels in train_dataloader:
    print(batch_features.shape)  # 預期輸出: torch.Size([32, 34, 29])
    print(batch_labels.shape)    # 預期輸出: torch.Size([32])
    break

torch.Size([32, 34, 29])
torch.Size([32])


In [ ]:
class MyCNN(nn.Module):
    def __init__(self, num_layers = 10, input_features=34, input_channels=29, output_features=34):
        super(MyCNN, self).__init__()
        
        self.conv_layers = nn.ModuleList()

        self.conv_layers.append(nn.Conv1d(in_channels=input_channels, out_channels=256, kernel_size=3, padding=1))
        
        for _ in range(1, num_layers - 1):
            self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1))
        
        self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=1, kernel_size=1, padding=0))
        
        self.activation = nn.LeakyReLU(negative_slope=0.01)

        self.fc = nn.Linear(input_features, output_features)
    def forward(self, x):
        x = x.squeeze(1)
        x = x.permute(0, 2, 1)

        for i, conv in enumerate(self.conv_layers):
            if(i < len(self.conv_layers) - 1):
                x = self.activation(conv(x))
            else:
                x = conv(x)
                
        x = x.squeeze(1)
        x = self.fc(x)
        print("x.shape:",x.shape)
        
        return x

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [46]:
model = MyCNN().to(device)
loss_criterion  = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)

In [47]:
current_loss = 0

start_time = time.time()
last_saved_time = start_time

for epoch in range(num_epochs):
    for train_data, value_data in train_dataloader:
        torch.autograd.set_detect_anomaly(True)
        channel1_tensor = train_data.unsqueeze(1)
        channel1_tensor = channel1_tensor.to(device)
        value_data = value_data.to(device)
        
        output = model(channel1_tensor)
        output = output.float()
        value_data = value_data.long()
        probabilities = F.softmax(output, dim=1)
        predicted_labels = torch.argmax(probabilities, dim=1)
        correct_predictions = (predicted_labels == value_data).sum().item()
        accuracy = (correct_predictions / value_data.size(0))*100
        
        loss = loss_criterion(output, value_data)
        
        current_loss = loss
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        clear_output()

        print(predicted_labels)
        print(value_data)
        print(f"Correct Predictions: {correct_predictions}")
        print(f"Total Predictions: {value_data.size(0)}")
        print(f"Accuracy: {accuracy:.4f}","%")
        print(f"At epoch: {epoch}, loss: {current_loss}")
        print(f"best accuracy: {best_accuracy}")
        
        if accuracy >= best_accuracy:
            best_accuracy = accuracy
            torch.save(model, f'E:/專題/discard_model/CNN_discard_model_{best_accuracy}.pth')
            print(f'模型在 epoch {epoch} 之後被儲存，正確率: {best_accuracy}')
        
        current_time = time.time()
        if current_time - last_saved_time >= 10800:
            timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime(current_time))
            model_path = f'E:/專題/discard_model/CNN_discard_model_{timestamp}.pth'
            torch.save(model, model_path)
            last_saved_time = current_time
            print(f'模型已基於時間間隔存儲，時間: {timestamp}')

tensor([28,  8, 30, 26, 32,  8, 31, 26, 32, 31, 28, 29, 32, 31, 26, 20, 31, 26,
        20, 32, 31, 20, 31, 20, 20, 26, 20, 26, 20, 26, 20, 32],
       device='cuda:0')
tensor([ 8,  5, 24,  9, 21, 23, 31, 28, 20,  5, 11,  0, 32,  4, 15,  4, 27,  8,
        31, 22, 30, 33, 31,  0, 15, 27, 25, 15, 18, 21,  8, 18],
       device='cuda:0')
Correct Predictions: 3
Total Predictions: 32
Accuracy: 9.3750 %
At epoch: 0, loss: 3.3935883045196533
best accuracy: 50


KeyboardInterrupt: 